In [79]:
import pickle
import os

from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
import sklearn
from sklearn import set_config


import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [80]:
set_config(transform_output = "pandas")

In [81]:
df = pd.read_csv(r'C:\Users\nilro\OneDrive\Escritorio\Capstone-Master-Data-Science\data_dsmarket\full_df.csv')

MemoryError: Unable to allocate 256. KiB for an array with shape (32768,) and data type int64

In [82]:
df.head()

,DATE,ID,ITEM,ITEM_ID,ITEM_CATEGORY,ITEM_CATEGORY_ID,ITEM_DEPARTMENT,ITEM_DEPARTMENT_ID,STORE_CODE,REGION,...,STORE,STORE_ID,SALES_COUNT,SELL_PRICE,REVENUE,EVENT_NEWYEAR,EVENT_NONE,EVENT_SUPERBOWL,EVENT_THANKSGIVING,EVENT_RAMADAN
0,2011-02-06,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,0.0,ACCESORIES,0.0,ACCESORIES_1,0.0,NYC_1,New York,...,Greenwich_Village,2.0,0,0.0,0.0,False,True,True,False,False
1,2011-02-13,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,0.0,ACCESORIES,0.0,ACCESORIES_1,0.0,NYC_1,New York,...,Greenwich_Village,2.0,0,0.0,0.0,False,True,False,False,False
2,2011-02-20,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,0.0,ACCESORIES,0.0,ACCESORIES_1,0.0,NYC_1,New York,...,Greenwich_Village,2.0,0,0.0,0.0,False,True,False,False,False
3,2011-02-27,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,0.0,ACCESORIES,0.0,ACCESORIES_1,0.0,NYC_1,New York,...,Greenwich_Village,2.0,0,0.0,0.0,False,True,False,False,False
4,2011-03-06,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,0.0,ACCESORIES,0.0,ACCESORIES_1,0.0,NYC_1,New York,...,Greenwich_Village,2.0,0,0.0,0.0,False,True,False,False,False


In [83]:
def load_data(path_data_file):
    '''
    Loads the csv file for training.
    '''
    df = pd.read_csv(path_data_file, index_col = 0)
    
    df = preprocess_df(df)
    
    return df

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8323770 entries, 0 to 8323769
Data columns (total 21 columns):
 #   Column              Dtype         
---  ------              -----         
 0   DATE                datetime64[ns]
 1   ID                  object        
 2   ITEM                object        
 3   ITEM_ID             float64       
 4   ITEM_CATEGORY       object        
 5   ITEM_CATEGORY_ID    float64       
 6   ITEM_DEPARTMENT     object        
 7   ITEM_DEPARTMENT_ID  float64       
 8   STORE_CODE          object        
 9   REGION              object        
 10  REGION_ID           float64       
 11  STORE               object        
 12  STORE_ID            float64       
 13  SALES_COUNT         int64         
 14  SELL_PRICE          float64       
 15  REVENUE             float64       
 16  EVENT_NEWYEAR       bool          
 17  EVENT_NONE          bool          
 18  EVENT_SUPERBOWL     bool          
 19  EVENT_THANKSGIVING  bool          
 20  EV

In [41]:
df['id'].nunique()

30490

In [42]:
df.duplicated().value_counts()

False    8323770
Name: count, dtype: int64

In [ ]:
ORDER_COLUMNS = [
    'DATE',
    'ID',
    'ITEM',
    'ITEM_ID',
    'CATEGORY',
    'CATEGORY_ID',
    'DEPARTMENT',
    'DEPARTMENT_ID',
    'STORE_CODE',
    'REGION',
    'REGION_ID',
    'STORE',
    'STORE_ID',
    'SALES_COUNT',
    'SELL_PRICE',
    'REVENUE',
    'EVENT_NEWYEAR',
    'EVENT_NONE',
    'EVENT_SUPERBOWL',
    'EVENT_THANKSGIVING',
    'EVENT_RAMADAN',
    
    
    
    
]

In [84]:
def preprocess_df(df):
    df.columns = map(str.upper, df.columns)
    df.drop(columns= 'Unnamed: 0', inplace = True)
    df['date'] = pd.to_datetime(df["date"], format = "%Y-%m-%d")
    
    df["CATEGORY_ID"] = OrdinalEncoder().fit_transform(df[["CATEGORY"]])
    df["DEPARTMENT_ID"] = OrdinalEncoder().fit_transform(df[["DEPARTMENT"]])
    df["REGION_ID"] = OrdinalEncoder().fit_transform(df[["REGION"]])
    df["STORE_ID"] = OrdinalEncoder().fit_transform(df[["STORE"]])
    df["ITEM_ID"] = OrdinalEncoder().fit_transform(df[["ITEM"]])
    
    df.rename(columns = {"CATEGORY":"ITEM_CATEGORY", "CATEGORY_ID":"ITEM_CATEGORY_ID", "DEPARTMENT":"ITEM_DEPARTMENT","DEPARTMENT_ID":"ITEM_DEPARTMENT_ID"}, inplace = True)
    df = df[ORDER_COLUMNS]
    
    return df


In [85]:
df['STORE'].value_counts()

STORE
Greenwich_Village    832377
Harlem               832377
Tribeca              832377
Brooklyn             832377
South_End            832377
Roxbury              832377
Back_Bay             832377
Midtown_Village      832377
Yorktown             832377
Queen_Village        832377
Name: count, dtype: int64

In [86]:
df[['STORE', 'STORE_CODE']]

,STORE,STORE_CODE
0,Greenwich_Village,NYC_1
1,Greenwich_Village,NYC_1
2,Greenwich_Village,NYC_1
3,Greenwich_Village,NYC_1
4,Greenwich_Village,NYC_1
...,...,...
8323765,Queen_Village,PHI_3
8323766,Queen_Village,PHI_3
8323767,Queen_Village,PHI_3
8323768,Queen_Village,PHI_3


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8323770 entries, 0 to 8323769
Data columns (total 21 columns):
 #   Column              Dtype         
---  ------              -----         
 0   DATE                datetime64[ns]
 1   ID                  object        
 2   ITEM                object        
 3   ITEM_ID             float64       
 4   ITEM_CATEGORY       object        
 5   ITEM_CATEGORY_ID    float64       
 6   ITEM_DEPARTMENT     object        
 7   ITEM_DEPARTMENT_ID  float64       
 8   STORE_CODE          object        
 9   REGION              object        
 10  REGION_ID           float64       
 11  STORE               object        
 12  STORE_ID            float64       
 13  SALES_COUNT         int64         
 14  SELL_PRICE          float64       
 15  REVENUE             float64       
 16  EVENT_NEWYEAR       bool          
 17  EVENT_NONE          bool          
 18  EVENT_SUPERBOWL     bool          
 19  EVENT_THANKSGIVING  bool          
 20  EV

In [88]:
df.head()

,DATE,ID,ITEM,ITEM_ID,ITEM_CATEGORY,ITEM_CATEGORY_ID,ITEM_DEPARTMENT,ITEM_DEPARTMENT_ID,STORE_CODE,REGION,...,STORE,STORE_ID,SALES_COUNT,SELL_PRICE,REVENUE,EVENT_NEWYEAR,EVENT_NONE,EVENT_SUPERBOWL,EVENT_THANKSGIVING,EVENT_RAMADAN
0,2011-02-06,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,0.0,ACCESORIES,0.0,ACCESORIES_1,0.0,NYC_1,New York,...,Greenwich_Village,2.0,0,0.0,0.0,False,True,True,False,False
1,2011-02-13,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,0.0,ACCESORIES,0.0,ACCESORIES_1,0.0,NYC_1,New York,...,Greenwich_Village,2.0,0,0.0,0.0,False,True,False,False,False
2,2011-02-20,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,0.0,ACCESORIES,0.0,ACCESORIES_1,0.0,NYC_1,New York,...,Greenwich_Village,2.0,0,0.0,0.0,False,True,False,False,False
3,2011-02-27,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,0.0,ACCESORIES,0.0,ACCESORIES_1,0.0,NYC_1,New York,...,Greenwich_Village,2.0,0,0.0,0.0,False,True,False,False,False
4,2011-03-06,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,0.0,ACCESORIES,0.0,ACCESORIES_1,0.0,NYC_1,New York,...,Greenwich_Village,2.0,0,0.0,0.0,False,True,False,False,False


Preprocessing